In [11]:
import sys, json, time,os
from types import SimpleNamespace
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\Joseph\generative-health-models")
SRC = PROJECT_ROOT / "src"

if not SRC.exists():
    raise FileNotFoundError(f"Expected folder not found: {SRC}")

# Ensure our local 'src' comes before site-packages (avoid clash with HuggingFace 'datasets')
os.chdir(PROJECT_ROOT)
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print("cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])

cwd: C:\Users\Joseph\generative-health-models
sys.path[0]: C:\Users\Joseph\generative-health-models\src


In [5]:
import numpy as np, os, sys

def check_norm_file(path, expected_channels):
    d = np.load(path)
    for key in ("mean", "std"):
        if key not in d:
            raise AssertionError(f"{path}: missing key '{key}'")
        if d[key].shape != (expected_channels,):
            raise AssertionError(
                f"{path}: {key} has shape {d[key].shape}, expected {(expected_channels,)}"
            )
        if not np.isfinite(d[key]).all():
            raise AssertionError(f"{path}: {key} contains non‑finite values")
    if (d["std"] <= 0).any():
        raise AssertionError(f"{path}: 'std' must be > 0 for all channels")
    print(f"{os.path.basename(path)} OK → mean={d['mean']}, std={d['std']}")
    return d["mean"], d["std"]

if __name__ == "__main__":
    m_low, s_low = check_norm_file("C:/Users/Joseph/generative-health-models/data/processed/tc_multigan_fold_S10_hp050_o2/norm_ecg.npz", 1)
    m_ecg, s_ecg = check_norm_file("C:/Users/Joseph/generative-health-models/data/processed/tc_multigan_fold_S10_hp050_o2/norm_low.npz", 2)

norm_ecg.npz OK → mean=[-5.614325e-09], std=[0.9999998]
norm_low.npz OK → mean=[-8.6496216e-10  1.1335446e-07], std=[0.9999582 0.9995736]


In [7]:
root = r"C:/Users/Joseph/generative-health-models/data/processed/tc_multigan_fold_S10"

def stats(path):
    x = np.load(os.path.join(root, path)).astype(np.float32)   # (N, T, C)
    flat = x.reshape(-1, x.shape[-1])
    mean = flat.mean(axis=0)
    std  = flat.std(axis=0)
    frac_gt3 = (np.abs(flat) > 3).mean(axis=0)
    print(f"{path}: mean={mean}, std={std}, frac|x|>3={frac_gt3}")

stats("train_X_low.npy")   # expect 2 channels (EDA, RESP)
stats("train_X_ecg.npy")   # expect 1 channel

train_X_low.npy: mean=[-5.7201682e-08 -8.6496216e-10], std=[0.99957025 0.9999582 ], frac|x|>3=[0.00770951 0.01551355]
train_X_ecg.npy: mean=[-5.614325e-09], std=[0.9999998], frac|x|>3=[0.02891105]


In [15]:
import sys, importlib, torch
sys.path.insert(0, r"C:\Users\Joseph\generative-health-models\src")

import models.tc_multigan as tcm
importlib.reload(tcm)

E = tcm.Embedder(hidden_dim=256, seq_length_low=120, seq_length_ecg=5280, latent_downsample=4)
R = tcm.Recovery(hidden_dim=256, seq_length_low=120, seq_length_ecg=5280, latent_downsample=4)

x_low = torch.randn(8, 120, 2)
x_ecg = torch.randn(8, 5280, 1)

h = E(x_low, x_ecg)              # -> (8, 256, 30)
low_hat, ecg_hat = R(h)          # -> expect (8, 120, 2), (8, 5280, 1)

print(h.shape, low_hat.shape, ecg_hat.shape)

torch.Size([8, 256, 30]) torch.Size([8, 120, 2]) torch.Size([8, 5280, 1])


In [19]:
import sys, importlib, torch
sys.path.insert(0, r"C:\Users\Joseph\generative-health-models\src")
import models.tc_multigan as tcm
importlib.reload(tcm)

E = tcm.Embedder(hidden_dim=256, seq_length_low=120, seq_length_ecg=5280, latent_downsample=4)
S = tcm.Supervisor(latent_dim=256, hidden_dim=256)

# fake batch
x_low = torch.randn(8, 120, 2)
x_ecg = torch.randn(8, 5280, 1)

# (B,H,T) -> (B,T,H) for S
h = E(x_low, x_ecg).transpose(1, 2)     # (8, 30, 256)
pred, tgt = S.predict_next(h)           # both (8, 29, 256)
print(h.shape, pred.shape, tgt.shape)

torch.Size([8, 30, 256]) torch.Size([8, 29, 256]) torch.Size([8, 29, 256])


In [20]:
from pathlib import Path
import numpy as np

PROJECT_ROOT = r"C:\Users\Joseph\generative-health-models"
root = Path(PROJECT_ROOT) / "data" / "processed"
fold = "tc_multigan_fold_s10"  # <-- adjust if needed

train_low = np.load(root / fold / "train_X_low.npy").astype(np.float64)  # (N, T_low, 2)
train_ecg = np.load(root / fold / "train_X_ecg.npy").astype(np.float64)  # (N, T_ecg, 1)

def chan_stats(x):  # x: (N, T, C)
    flat = x.reshape(-1, x.shape[-1])
    mu = flat.mean(axis=0)
    sd = flat.std(axis=0)          # ddof=0 to match np.std default
    # safety clamp for zero std
    sd = np.where(sd <= 0, 1.0, sd)
    return mu.astype(np.float32), sd.astype(np.float32)

mu_low, sd_low = chan_stats(train_low)
mu_ecg, sd_ecg = chan_stats(train_ecg)

out_low = root / fold / "stats_low_train_raw.npz"
out_ecg = root / fold / "stats_ecg_train_raw.npz"
np.savez(out_low, mean=mu_low, std=sd_low)
np.savez(out_ecg, mean=mu_ecg, std=sd_ecg)
print("Saved:", out_low, out_ecg)
print("low mean/std:", mu_low.tolist(), sd_low.tolist())
print("ecg mean/std:", mu_ecg.tolist(), sd_ecg.tolist())

Saved: C:\Users\Joseph\generative-health-models\data\processed\tc_multigan_fold_s10\stats_low_train_raw.npz C:\Users\Joseph\generative-health-models\data\processed\tc_multigan_fold_s10\stats_ecg_train_raw.npz
low mean/std: [-9.771369491318183e-09, 7.395823953260106e-10] [1.0, 0.9999997019767761]
ecg mean/std: [-5.592353691241669e-09] [1.0000001192092896]


In [21]:
from pathlib import Path
import numpy as np

root = Path(r"C:\Users\Joseph\generative-health-models") / "data" / "processed" / "tc_multigan_fold_s10"

def flat_std(x): return x.reshape(-1, x.shape[-1]).std(axis=0)

val_low_raw = np.load(root / "test_X_low.npy").astype(np.float64)
val_ecg_raw = np.load(root / "test_X_ecg.npy").astype(np.float64)
trn_low_raw = np.load(root / "train_X_low.npy").astype(np.float64)
trn_ecg_raw = np.load(root / "train_X_ecg.npy").astype(np.float64)

print("std_val_raw low:", flat_std(val_low_raw))
print("std_trn_raw low:", flat_std(trn_low_raw))
print("expected val std after train-norm (low):", flat_std(val_low_raw) / flat_std(trn_low_raw))

print("std_val_raw ecg:", flat_std(val_ecg_raw))
print("std_trn_raw ecg:", flat_std(trn_ecg_raw))
print("expected val std after train-norm (ecg):", flat_std(val_ecg_raw) / flat_std(trn_ecg_raw))

std_val_raw low: [0.14074133 0.91131198]
std_trn_raw low: [1.00000005 0.99999973]
expected val std after train-norm (low): [0.14074132 0.91131223]
std_val_raw ecg: [0.53762832]
std_trn_raw ecg: [1.00000012]
expected val std after train-norm (ecg): [0.53762825]


In [22]:
from pathlib import Path
import numpy as np

fold = Path(r"C:\Users\Joseph\generative-health-models\data\processed\tc_multigan_fold_S10")
cond_p = fold / "test_cond.npy"
cond = np.load(cond_p)

print("test_cond.npy shape:", cond.shape)

# If one-hot over time (N,120,K), compute class prevalence across windows & time
if cond.ndim == 3:
    K = cond.shape[-1]
    prev = cond.mean(axis=(0,1))
    print("K (num classes) =", K)
    print("Class prevalence (argmax order):", prev)
# If one-hot (N,K), compute prevalence across windows
elif cond.ndim == 2:
    K = cond.shape[-1]
    prev = cond.mean(axis=0)
    print("K (num classes) =", K)
    print("Class prevalence (argmax order):", prev)
# If already integer labels (N,)
else:
    uniq, cnt = np.unique(cond, return_counts=True)
    print("Integer labels found:", dict(zip(uniq.tolist(), cnt.tolist())))

test_cond.npy shape: (194,)
Integer labels found: {1: 77, 2: 46, 3: 23, 4: 48}


In [24]:
import numpy as np
from pathlib import Path

REAL_SPLIT_DIR = Path(r"C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split")

for name in ["baseline","stress","amusement"]:
    p = REAL_SPLIT_DIR / f"real_{name}.npz"
    d = np.load(p)
    print("\n", p)
    print(" keys:", list(d.keys()))
    for k in d.keys():
        v = d[k]
        try:
            print(f"  {k}: shape={v.shape}, dtype={v.dtype}")
        except Exception:
            print(f"  {k}: (non-array)")


 C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split\real_baseline.npz
 keys: ['X', 'y']
  X: shape=(23, 5250, 3), dtype=float32
  y: shape=(23,), dtype=int32

 C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split\real_stress.npz
 keys: ['X', 'y']
  X: shape=(46, 5250, 3), dtype=float32
  y: shape=(46,), dtype=int32

 C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split\real_amusement.npz
 keys: ['X', 'y']
  X: shape=(77, 5250, 3), dtype=float32
  y: shape=(77,), dtype=int32


In [25]:
import numpy as np
from pathlib import Path

REAL_SPLIT_DIR = Path(r"C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split")

for name, expected in [("baseline",0),("stress",1),("amusement",2)]:
    d = np.load(REAL_SPLIT_DIR / f"real_{name}.npz")
    labs = d["labels"] if "labels" in d else d["y"]
    u = np.unique(labs)
    print(f"{name}: unique labels -> {u}  (expected only {expected})")

baseline: unique labels -> [0]  (expected only 0)
stress: unique labels -> [1]  (expected only 1)
amusement: unique labels -> [2]  (expected only 2)


In [26]:
from pathlib import Path
import numpy as np

SRC = Path(r"C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split")
DST = Path(r"C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split_shadow_wesad")
DST.mkdir(parents=True, exist_ok=True)

for name in ["baseline","stress","amusement"]:
    d = np.load(SRC / f"real_{name}.npz")
    # convert X/y → signals/labels (+channels) if needed
    if {"signals","labels"}.issubset(d.files):
        signals = d["signals"]; labels = d["labels"]
        channels = d["channels"] if "channels" in d.files else np.array(["ECG","Resp","EDA"], dtype=object)
    else:
        signals = d["X"]; labels = d["y"]
        channels = np.array(["ECG","Resp","EDA"], dtype=object)
    np.savez_compressed(DST / f"real_{name}.npz",
                        signals=signals.astype(np.float32, copy=False),
                        labels=labels.astype(np.int32,  copy=False),
                        channels=channels)
    print("shadow wrote:", DST / f"real_{name}.npz", signals.shape, labels.shape)

shadow wrote: C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split_shadow_wesad\real_baseline.npz (23, 5250, 3) (23,)
shadow wrote: C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split_shadow_wesad\real_stress.npz (46, 5250, 3) (46,)
shadow wrote: C:\Users\Joseph\generative-health-models\results\evaluation\real_3class_split_shadow_wesad\real_amusement.npz (77, 5250, 3) (77,)
